In [1]:
import pycaret
from pycaret.regression import *

In [18]:
import pandas as pd
from pandas import to_datetime

df = pd.read_csv('../transaction_data_version3.csv')
product_df = pd.read_csv('../product.csv')
product_df1 = product_df[['PRODUCT_ID','SUB_COMMODITY_DESC']]
df_merge = df.merge(product_df1,on='PRODUCT_ID')
df_merge.groupby('SUB_COMMODITY_DESC')['QUANTITY'].sum().sort_values()
df_milk = df_merge.query('SUB_COMMODITY_DESC == "FLUID MILK WHITE ONLY"')
df_milk_1 = df_milk[['DATE','QUANTITY']]
df_milk_1 = df_milk_1.groupby('DATE')['QUANTITY'].sum().reset_index()
df_milk_1['DATE']= to_datetime(df_milk_1['DATE'])
df_milk_1['DATE'] = df_milk_1['DATE'].dt.strftime('%Y-%m')
df_milk_1 = df_milk_1.groupby('DATE')['QUANTITY'].sum().reset_index()
df_milk_1['DATE'] = pd.to_datetime(df_milk_1['DATE'])
df_milk_1

,DATE,QUANTITY
0,2019-07-01,8
1,2019-08-01,724
2,2019-09-01,1439
3,2019-10-01,2759
4,2019-11-01,3622
5,2019-12-01,3764
6,2020-01-01,3862
7,2020-02-01,3768
8,2020-03-01,4618
9,2020-04-01,4345


In [20]:
# create 12 month moving average
df_milk_1['MA12'] = df_milk_1['QUANTITY'].rolling(2).mean()
# plot the data and MA
import plotly.express as px
fig = px.line(df_milk_1, x="DATE", y=["QUANTITY", "MA12"], template = 'plotly_dark')
fig.show()

In [21]:
# extract month and year from dates
df_milk_1['Month'] = [i.month for i in df_milk_1['DATE']]
df_milk_1['Year'] = [i.year for i in df_milk_1['DATE']]
# create a sequence of numbers
df_milk_1['Series'] = np.arange(1,len(df_milk_1)+1)
# drop unnecessary columns and re-arrange
df_milk_1.drop(['DATE', 'MA12'], axis=1, inplace=True)
df_milk_1 = df_milk_1[['Series', 'Year', 'Month', 'QUANTITY']] 
# check the head of the dataset
df_milk_1.head()

,Series,Year,Month,QUANTITY
0,1,2019,7,8
1,2,2019,8,724
2,3,2019,9,1439
3,4,2019,10,2759
4,5,2019,11,3622


In [34]:
# split data into train-test set
train = df_milk_1.query('(Year < 2021 & Month <= 12) or (Year == 2021 & Month < 3)')
test = df_milk_1.query('Year == 2021 & Month >=3')

# check shape
train.shape, test.shape

((20, 4), (5, 4))

In [35]:
s = setup(data = train, test_data = test, target = 'QUANTITY', 
          fold_strategy = 'timeseries', numeric_features = ['Year', 'Series'], 
          fold = 3, transform_target = True, session_id = 123)

,Description,Value
0,session_id,123
1,Target,QUANTITY
2,Original Data,"(20, 4)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(20, 14)"


In [36]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dt,Decision Tree Regressor,414.2667,269588.1333,516.2904,-0.5994,0.0000,0.0992,0.0133
gbr,Gradient Boosting Regressor,476.8707,321034.4251,554.0599,-1.0801,0.0000,0.1137,0.0200
ada,AdaBoost Regressor,736.5248,817821.9053,820.0249,-5.0220,0.0000,0.1769,0.0333
et,Extra Trees Regressor,820.6061,1094023.0312,933.7227,-5.3183,0.0000,0.1860,0.1667
rf,Random Forest Regressor,946.8416,1358344.9556,989.3797,-9.4793,0.0000,0.2272,0.1700
lasso,Lasso Regression,972.8054,1306779.4950,1140.2904,-6.3122,0.0000,0.2392,0.6133
knn,K Neighbors Regressor,1265.8654,2875916.2423,1297.6956,-22.1178,0.0000,0.3050,0.0533
en,Elastic Net,1517.0156,3185359.2261,1765.8213,-18.7582,0.0000,0.3618,0.0100
llar,Lasso Least Angle Regression,1818.7632,4062619.7884,1885.7811,-27.8171,0.0000,0.4238,0.7533
lightgbm,Light Gradient Boosting Machine,1818.7632,4062619.7231,1885.7811,-27.8171,0.0000,0.4238,0.5467


In [37]:
prediction_holdout = predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Decision Tree Regressor,1144.4000,2718824.0000,1648.8857,-0.6861,0.6175,0.6736


In [43]:
# generate predictions on the original dataset
predictions = predict_model(best, data=df_milk_1)
# add a date column in the dataset
predictions['DATE'] = pd.date_range(start='2019-07-01', end = '2021-07-01', freq = 'MS')
# line plot
fig = px.line(predictions, x='DATE', y=["QUANTITY", "Label"], template = 'plotly_dark')
# add a vertical rectange for test-set separation
fig.add_vrect(x0="2019-07-01", x1="2021-07-01", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()